In [ ]:
#https://www.youtube.com/watch?v=7_Js8h709Dw
# normalizing, removing stationarity, volatility, seasonality

In [1]:
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
import pandas as pd
new_df = pd.read_excel('mw_dataset - with CODED MOH annoucements - recategorized.xlsx',index_col='date',parse_dates=['date'])

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
new_df.columns

Index(['Unnamed: 0', 'SN', 'facebook_count', 'facebook_fear_count',
       'facebook_fear_percentage', 'facebook_fear_intensity',
       'facebook_anger_count', 'facebook_anger_percentage',
       'facebook_anger_intensity', 'facebook_happy_count',
       ...
       'announcement_safedist', 'announcement_shn', 'announcement_speech',
       'announcement_speech_min_transcript',
       'announcement_speech_min_transcript_pm', 'announcement_tracetogether',
       'announcement_travel', 'announcement_travelborder',
       'announcement_travel_bubble', 'announcement_vaccination'],
      dtype='object', length=109)

In [ ]:
###normalizing the data

In [535]:
avg, dev = new_df['announcement_vaccination'].mean(), new_df['announcement_vaccination'].std()

In [536]:
fb_series = (new_df['announcement_vaccination'] - avg) / dev

In [538]:
##Take first difference to remove stationarity

fb_series = fb_series.diff()
fb_series.fillna(0, inplace=True)

In [540]:
monthly_volatility = fb_series.groupby(fb_series.index.month).std()

In [541]:
monthly_volatility

date
1     1.237355
2     1.004576
3     1.898470
4     1.605761
5     2.086551
6     1.289061
7     0.474617
8     0.583898
9     0.000000
10    0.000000
11    0.000000
12    2.367808
Name: announcement_vaccination, dtype: float64

In [542]:
fb_monthly_int = fb_series.index.map(lambda d: monthly_volatility.loc[d.month])

In [543]:
fb_monthly_int

Float64Index([ 1.237354939227018,  1.237354939227018,  1.237354939227018,
               1.237354939227018,  1.237354939227018,  1.237354939227018,
               1.004575819354692,  1.898469850801044,  1.605760775705036,
              2.0865509830240985,
              ...
              0.4746174627002611, 0.4746174627002611, 0.4746174627002611,
              0.4746174627002611, 0.4746174627002611, 0.4746174627002611,
              0.4746174627002611, 0.4746174627002611, 0.4746174627002611,
              0.4746174627002611],
             dtype='float64', name='date', length=552)

In [544]:
#removing volatility by taking series and dividing by monthly std deviation
fb_series = fb_series / fb_monthly_int

In [545]:
##fb_series.plot()
#plt.axhline(0, linestyle='--',color='k', alpha=0.3)

In [546]:
##Removing seasonality by subtracting monthly means
month_avgs = fb_series.groupby(fb_series.index.month).mean()

In [547]:
month_avgs

date
1    -0.068343
2     0.000000
3     0.133631
4    -0.122881
5     0.030396
6    -0.051024
7     0.133631
8    -0.164399
9          NaN
10         NaN
11         NaN
12    0.000000
dtype: float64

In [548]:
fb_month_avg = fb_series.index.map(lambda d: month_avgs.loc[d.month])

In [549]:
fb_month_avg

Float64Index([-0.06834301594649277, -0.06834301594649277, -0.06834301594649277,
              -0.06834301594649277, -0.06834301594649277, -0.06834301594649277,
                               0.0,  0.13363062095621223, -0.12288089802615826,
              0.030396298376268942,
              ...
                0.1336306209562122,   0.1336306209562122,   0.1336306209562122,
                0.1336306209562122,   0.1336306209562122,   0.1336306209562122,
                0.1336306209562122,   0.1336306209562122,   0.1336306209562122,
                0.1336306209562122],
             dtype='float64', name='date', length=552)

In [550]:
fb_series = fb_series - fb_month_avg

In [551]:
##fb_series.plot()
#plt.axhline(0, linestyle='--',color='k', alpha=0.3)

In [552]:
###doing the cleaning for the whole dataset

In [553]:
#new_df.drop(['normalized_facebook_fear_count'], axis = 1, inplace=True)

In [554]:
new_df['normalized_announcement_vaccination'] = fb_series

In [555]:
new_df['normalized_announcement_vaccination'].fillna(0, inplace=True)

In [556]:
new_df.head()

,Unnamed: 0,SN,facebook_count,facebook_fear_count,facebook_fear_percentage,facebook_fear_intensity,facebook_anger_count,facebook_anger_percentage,facebook_anger_intensity,facebook_happy_count,facebook_happy_percentage,facebook_happy_intensity,facebook_sad_count,facebook_sad_percentage,facebook_sad_intensity,tweet_count,tweet_fear_count,tweet_fear_percentage,tweet_fear_intensity,tweet_anger_count,tweet_anger_percentage,tweet_anger_intensity,tweet_happy_count,tweet_happy_percentage,tweet_happy_intensity,tweet_sad_count,tweet_sad_percentage,tweet_sad_intensity,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths,Announcement_count,rss_count,normalized_facebook_fear_intensity,normalized_facebook_count,normalized_facebook_fear_count,normalized_facebook_fear_percentage,normalized_facebook_anger_count,normalized_facebook_anger_percentage,normalized_facebook_anger_intensity,normalized_facebook_happy_count,normalized_facebook_happy_percentage,normalized_facebook_happy_intensity,normalized_facebook_sad_count,normalized_facebook_sad_percentage,normalized_facebook_sad_intensity,normalized_tweet_count,normalized_tweet_fear_count,normalized_tweet_fear_percentage,normalized_tweet_fear_intensity,normalized_tweet_anger_count,normalized_tweet_anger_percentage,normalized_tweet_anger_intensity,normalized_tweet_happy_count,normalized_tweet_happy_percentage,normalized_tweet_happy_intensity,normalized_tweet_sad_count,normalized_tweet_sad_percentage,normalized_tweet_sad_intensity,normalized_new_cases,normalized_cumulative_cases,normalized_new_deaths,normalized_cumulative_deaths,normalized_announcement_count,normalized_rss_count,mindline_main,mindline_protocol_crisis,mindline_protocol_mild,mindline_protocol_moderate,mindline_protocol_well,mindline_protocol_total,normalized_mindline_main,normalized_mindline_protocol_crisis,normalized_mindline_protocol_mild,normalized_mindline_protocol_moderate,normalized_mindline_protocol_well,normalized_mindline_protocol_total,mindline_direct,mindline_google,mindline_dng,normalized_mindline_direct,normalized_mindline_google,normalized_mindline_dng,announcement_advisory,announcement_caseupdate,announcement_circuitbreaker,announcement_disciplinarymeasures,announcement_falsehood_correction,announcement_global_update,announcement_govt_support,announcement_migrant_worker,announcement_no_new_case,announcement_others,announcement_precautionary_measures,announcement_precautionary_measures_religion,announcement_precautionary_measures_school,announcement_precautionary_measures_sports,announcement_reopening,announcement_safedist,announcement_shn,announcement_speech,announcement_speech_min_transcript,announcement_speech_min_transcript_pm,announcement_tracetogether,announcement_travel,announcement_travelborder,announcement_travel_bubble,announcement_vaccination,normalized_announcement_advisory,normalized_announcement_caseupdate,normalized_announcement_circuitbreaker,normalized_announcement_disciplinarymeasures,normalized_announcement_falsehood_correction,normalized_announcement_global_update,normalized_announcement_govt_support,normalized_announcement_migrant_worker,normalized_announcement_no_new_case,normalized_announcement_others,normalized_announcement_precautionary_measures,normalized_announcement_precautionary_measures_religion,normalized_announcement_precautionary_measures_school,normalized_announcement_precautionary_measures_sports,normalized_announcement_reopening,normalized_announcement_safedist,normalized_announcement_shn,normalized_announcement_speech,normalized_announcement_speech_min_transcript,normalized_announcement_speech_min_transcript_pm,normalized_announcement_tracetogether,normalized_announcement_travel,normalized_announcement_travelborder,normalized_announcement_travel_bubble,normalized_announcement_vaccination
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-27,0,27,126,30,0.000000,0.532900,46,36.507937,1.0000

In [557]:
new_df.to_excel('till july dataset coded moh announcements recategorized normalized FINAL.xlsx')

In [ ]:
#redo ffp?